# Chapter11. 인터페이스: 프로토콜에서 ABC까지
11장의 주제는 인터페이스다. 덕 타이핑의 상징인 동적 프로토콜에서부터 인터페이스를 명시하고 구현이 인터페이스에 따르는 지 검증하는 추상 베이스 클래스(abstract base class: ABC)에 이르기까지 다양한 곳에서 인터페이스가 사용된다. 

파이썬 커뮤니티에서 인터페이스를 전통적으로 약간 느슨하게 이해해온 방식을 살펴보자. (두 개의 예제를 통해 덕 타이핑의 동적 성질을 이해)


## 11. 1  파이썬 문화에서의 인터페이스와 프로토콜
파이썬은 ABC가 소개되기 전에 이미 성공을 거두었고, 기존 코드 대부분은 ABC를 전혀 사용하지 않는다. 필자는 1장부터 줄곧 덕 타이핑과 프로토콜에 대해 이야기했다. 
동적 자료형 언어에서는 인터페이스가 어떻게 작동할까? 우선 기본적으로 파이썬 언어에는 interface라는 키워드가 없지만, ABC에 상관없이 모든 클래스는 인터페이스를 가지고 있다. 클래스가 상속하거나 구현한 공개 속성(메서드나 데이터 속성)드르이 집합이 인터페이스다. 여기에는 `__getitem__()`이나 `__add__()`같은 특별 메서드도 포함된다.

In [1]:
# vector2d_v0.py
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    # more methods follow (omitted in this listing)

해당 버전에서 x와 y를 읽기 전용 프로퍼티로 변경했다. 이것은 상당한 리팩토링이지만, 인터페이스의 핵심은 변하지 않았다. 사용자는 여전히 my_vector.x와 my_vector.y로 값을 읽을 수 있기 때문이다.

In [2]:
# vector2d_v3.py
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))


인터페이스는 '시스템에서 어떤 역할을 할 수 있게 해주는 객체의 공개 메서드의 일부'라고 설명을 보충할 수 있다. 바로 이것이 파이썬 문서에서 특정 클래스를 지정하지 않고, 'file과 같은 객체', 혹은 '반복형'이라고 말할 때 의미하는 것이다. 어떤 역할을 완수하기 위한 메서드 집합으로서의 인터페이스를 스몰토크에서는 프로토콜이라고 불렀으며, 이 용어는 다른 동적 언어 커뮤니티에도 퍼져나갔다. 프로토콜은 상속과 무관하다. 프로토콜은 인터페이스지만 비공식적이다.
시퀀스 프로토콜은 파이썬에서 가장 핵심적인 인터페이스 중 하나다.

## 11.2 파이썬은 시퀀스를 찾아낸다.
파이썬 데이터 모델은 가능한 한 많이 핵심 프로토콜과 협업하겠다는 철학을 가지고 있다. 시퀀스의 경우, 가장 단순한 객체를 사용하는 경우에도 파이썬은 최선을 다한다.

In [19]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

f = Foo()
f[1]
for i in f: print(i)

0
10
20


In [21]:
20 in f

True

In [22]:
15 in f

False

`__iter__()`메서드는 구현하지 않았지만, 대체 수단인 `__getitem__()`을 구현했기 때문에 Foo를 반복할 수 있다. 파이썬 인터프리터는 0부터 시작하는 정수 인덱스로 getitiem 메서드를 호출하여 객체 반복을 시도하기 때문이다. 파이썬은 충분히 똑똑하기 때문에 `__contains__()` 메서드가 구현되어 있지 않아도 객체 전체를 조사해서 항목을 찾아내 `in` 연산자도 작동시킬 수 있다.

In [7]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])


class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

## 11.3 런타임에 프로토콜을 구현하는 멍키 패칭
위의 FrenchDeck은 카드를 섞을 수 없다는 커다란 결함이 있다. 몇년 전 필자가 처음 FrenchDeck을 작성했을 때는 shuffle() 메서드를 구현했었는데, 파이썬을 알고 난 이후는 시퀀스처럼 작동하는 클래스라면 shuffle을 직접 구현할 필요가 없다는 것을 깨달았다. random.shuffle() 함수 문서에서 설명하는 것처럼 random.shuffle() 함수가 시퀀스 객체안의 항목을 섞어주기 때문이다.

In [9]:
from random import shuffle
l = list(range(10))
shuffle(l)
print(l)

# 하지만 FrenchDeck 객체를 섞고 싶다면?
from random import shuffle

deck = FrenchDeck()
shuffle(deck)

[1, 2, 4, 6, 7, 3, 9, 0, 5, 8]


TypeError: 'FrenchDeck' object does not support item assignment

'FrenchDeck' 클래스는 불변 시퀀스 프로토콜만 구현하고 있다. 가변 시퀀스는 `__setitem__()` 메서드도 지원해야 한다.

In [15]:
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='3', suit='clubs'),
 Card(rank='4', suit='hearts'),
 Card(rank='8', suit='spades'),
 Card(rank='10', suit='spades'),
 Card(rank='J', suit='diamonds')]

## 11.4 알렉스 마르텔리의 물새
파이썬의 일반적인 프로토콜형 인터페이스를 살펴본 뒤 ABC를 살펴보자. 그러나 예제와 세부사항을 파고들기 전에, ABC가 파이썬에 큰 도움이 되는 이유에 대해 알렉스가 알려준댄다

In [4]:
class Struggle:
    def __len__(self): return 23


from collections import abc
isinstance(Struggle(), abc.Sized)

True

## 11.5 ABC상속하기
직접 ABC를 만들기 전에 collections.MutableSequence라는 ABC를 활용해보자. FrenchDeck2를 그렇게 만들보자

In [41]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits 
                                        for rank in self.ranks]

    # def __len__(self):
    #     return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):     # 카드를 섞기위해서 __setitem__() 메서드만 있으면 된다.
        self._cards[position] = value

    def __delitem__(self, position):            # 그러나 MutableSequence 클래스를 상속했으므로, 이 클래스의 추상메서드인 __delitem__()도 구현해야 한다.
        del self._cards[position]

    def insert(self, position, value):          # 그리고 MutableSequence의 세 번째 추상 메서드인 insert() 도 구현해야 한다.
        self._cards.insert(position, value)

## 11.6 표준 라이브러리의 ABC
파이썬 2.6 이후 표준 라이브러리에 ABC가 포함되었다. numbers와 io 패키지에서도 볼 수 있지만, 대부분의 ABC는 collections.abc 모듈에 정의되어 있으며, 이 모듈에 정의된 ABC들이 가장 많이 사용된다.

### 11.6.1 collections.abc의 ABC
표준 라이브러리에 abc라는 모듈은 두개가 있다. 여기서는 collections.abc에 대해 설명한다. 


### 11.6.2 ABC의 숫자탑
numbers 패키지는 소위 '수ㅅ자탑'이라고 하는 것을 정의한다(말 그대로 ABC들이 선형 계층구조로 되어 있다.) . 
Number가 최상위 슈퍼클래스며, 그 밑으로
- Number
- Complex
- Real
- Rational
- Integral
따라서 정수형인지 검사해야 하는 경우 isinstance(x, numbers.Integral)을 이용해서 int형, bool형(int형의 서브클래스)

## 11.7 ABC의 정의와 사용
ABC를 생성하는 일을 정당화하기 위해, 프레임워크를 확장해야 하는 상황을 만들어보자.

In [45]:
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """iterable의 항목들을 추가한다."""

    @abc.abstractmethod
    def pick(self):
        """무작위로 항목을 하나 제거하고 반환한다.
        객체가 비어 있을 때 이 메서드를 실행하면 'LookupError'가 발생한다.
        """

    def loaded(self):
        """최소 한 개의 항목이 있으면 True를, 아니면 False를 반환한다."""
        return bool(self.inspect())

    def inspect(self):
        """현재 안에 있는 항목들로 구성된 튜플을 정렬해 반환한다."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [46]:
from tombola import Tombola

class Fake(Tombola):
    def pick(self):
        return 13

In [47]:
Fake

__main__.Fake

In [48]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

### 11.7.2 Tombola ABC 상속하기

In [49]:
import random
from tombola import Tombola

class BingoCage(Tombola):

    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

In [ ]:
import random

from tombola import Tombola

class LotteryBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)
    
    def load(self, iterable):
        self._balls.extend(iterable)
    
    def pick(self):
        try:
            position = random
